In [ ]:
!pip install tensorflow

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import tensorflow as tf
from sklearn.metrics import classification_report
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
from keras.preprocessing import image
from tensorflow.keras.utils import load_img
import numpy as np

#### Definindo hiperparâmetro

In [ ]:
batch_size = 64
epochs = 50
input_shape = (150, 150, 3)  # Tamanho das imagens de entrada

###geradores de dados para pré-processamento

In [ ]:
data_generator = ImageDataGenerator(rescale=1./255, validation_split=(0.3))
path = '/content/drive/MyDrive/Pasta sem nome/train'

train_generator = data_generator.flow_from_directory(
    path,
    target_size=input_shape[:2],
    shuffle=True,
    batch_size=batch_size,
    class_mode='categorical',
    subset='training')

validation_generator = data_generator.flow_from_directory(
    path,
    target_size=input_shape[:2],
    shuffle=True,
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation')

Found 3664 images belonging to 2 classes.
Found 1568 images belonging to 2 classes.


###Construindo o modelo da CNN (REDES NEURAIS CONVOLUCIONAIS)

In [ ]:
filepeath = 'transferlearning_weigths.hdf5'
checkpoint = ModelCheckpoint(filepeath, monitor='val_loss', verbose=1, save_best_only=True, mode='max')

In [ ]:
lr_reduce = ReduceLROnPlateau(monitor= 'val_loss', factor=0.1, min_delta= 0.001, patience= 3, verbose=1)

In [ ]:
early_stop = EarlyStopping(monitor= 'val_loss',min_delta= 0.001, patience=9, mode='max', verbose=1)

In [ ]:
callbacks = [checkpoint, lr_reduce, early_stop]

In [ ]:
model = Sequential()

model.add(Conv2D(64, (3, 3), activation='relu', input_shape=input_shape, ))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.3))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.3))

model.add(Flatten())

model.add(Dense(256, activation='relu'))
model.add(Dropout(0.4))

model.add(Dense(2, activation='sigmoid'))

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           (None, 148, 148, 64)      1792      
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 74, 74, 64)       0         
 2D)                                                             
                                                                 
 dropout_3 (Dropout)         (None, 74, 74, 64)        0         
                                                                 
 conv2d_3 (Conv2D)           (None, 72, 72, 128)       73856     
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 36, 36, 128)      0         
 2D)                                                             
                                                                 
 dropout_4 (Dropout)         (None, 36, 36, 128)      

### COMPILAÇÃO DO MODELO

In [ ]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

### TREINO DO MODELO

In [ ]:
model.fit(train_generator,
          steps_per_epoch=train_generator.samples//batch_size,
          epochs=epochs,
          validation_data=validation_generator,
          validation_steps=validation_generator.samples//batch_size,
          callbacks = callbacks)

Epoch 1/50
57/57 [==============================] - ETA: 0s - loss: 1.0008 - accuracy: 0.7722
Epoch 1: val_loss improved from -inf to 0.31966, saving model to transferlearning_weigths.hdf5
57/57 [==============================] - 558s 10s/step - loss: 1.0008 - accuracy: 0.7722 - val_loss: 0.3197 - val_accuracy: 0.8711 - lr: 0.0010
Epoch 2/50
57/57 [==============================] - ETA: 0s - loss: 0.1832 - accuracy: 0.9306
Epoch 2: val_loss did not improve from 0.31966
57/57 [==============================] - 542s 10s/step - loss: 0.1832 - accuracy: 0.9306 - val_loss: 0.1671 - val_accuracy: 0.9408 - lr: 0.0010
Epoch 3/50
57/57 [==============================] - ETA: 0s - loss: 0.1124 - accuracy: 0.9603
Epoch 3: val_loss did not improve from 0.31966
57/57 [==============================] - 544s 10s/step - loss: 0.1124 - accuracy: 0.9603 - val_loss: 0.1417 - val_accuracy: 0.9525 - lr: 0.0010
Epoch 4/50
57/57 [==============================] - ETA: 0s - loss: 0.0841 - accuracy: 0.9706
Epo

###AVALIÇÃO DO MODELO

In [ ]:
test_loss, test_accuracy = model.evaluate(validation_generator, steps=len(validation_generator))
print('Teste de Perda:', test_loss)
print('Teste de  Acuracia:', test_accuracy)

25/25 [==============================] - 67s 3s/step - loss: 0.2036 - accuracy: 0.9247
Teste de Perda: 0.20363546907901764
Teste de  Acuracia: 0.9247449040412903


In [ ]:
prediction = model.predict(validation_generator)
if prediction == 1:
    print('Pneumonia detectada.')
else:
    print('Pneumonia não detectada.')

{'loss': <tf.Tensor: shape=(), dtype=float32, numpy=0.20363547>,
 'accuracy': <tf.Tensor: shape=(), dtype=float32, numpy=0.9247449>}